**Importing the Dependencies**

In [25]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

**Device Configuration**

In [26]:
# check for CUDA availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'using device: {device}')

using device: cpu


**Data Collection and preprocessing**

In [27]:
# load the breast cancer dataset
data = load_breast_cancer()
X = data.data
y = data.target

In [28]:
print(X)

[[1.799e+01 1.038e+01 1.228e+02 ... 2.654e-01 4.601e-01 1.189e-01]
 [2.057e+01 1.777e+01 1.329e+02 ... 1.860e-01 2.750e-01 8.902e-02]
 [1.969e+01 2.125e+01 1.300e+02 ... 2.430e-01 3.613e-01 8.758e-02]
 ...
 [1.660e+01 2.808e+01 1.083e+02 ... 1.418e-01 2.218e-01 7.820e-02]
 [2.060e+01 2.933e+01 1.401e+02 ... 2.650e-01 4.087e-01 1.240e-01]
 [7.760e+00 2.454e+01 4.792e+01 ... 0.000e+00 2.871e-01 7.039e-02]]


In [29]:
print(y[:5])

[0 0 0 0 0]


In [30]:
# split the dataset into training and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [31]:
print(X.shape)
print(X_train.shape)
print(X_test.shape)

(569, 30)
(455, 30)
(114, 30)


In [32]:
# standardize the data using Standard scaler
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [33]:
type(X_train)

numpy.ndarray

In [34]:
# convert data to Pytorch tensors and move it to GPU
X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train = torch.tensor(y_train, dtype=torch.float32).to(device)
X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test = torch.tensor(y_test, dtype=torch.float32).to(device)

**Neural Network Architecture**

In [35]:
# define the neural architecture

class NeuralNet(nn.Module):

  def __init__(self, input_size, hidden_size, output_size):
    super(NeuralNet, self).__init__()
    self.fc1 = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(hidden_size, output_size)
    self.sigmoid = nn.Sigmoid()

  def forward(self, x):
    out = self.fc1(x)
    out = self.relu(out)
    out = self.fc2(out)
    out = self.sigmoid(out)
    return out

In [36]:
#define hyperparameters
input_size = X_train.shape[1]
hidden_size = 64
output_size = 1
learning_rate = 0.001
num_epochs = 100

In [37]:
# initialize the neural network and move it the GPU
model = NeuralNet(input_size, hidden_size, output_size).to(device)

In [38]:
# define loss and the optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

**Training the Neural Network**

In [39]:
# training the model
for epoch in range(num_epochs):
  model.train()
  optimizer.zero_grad()
  outputs = model(X_train)
  loss = criterion(outputs,y_train.view(-1,1))
  loss.backward()
  optimizer.step()

  # calculate accuracy
  with torch.no_grad():
    predicted = outputs.round()
    correct = (predicted == y_train.view(-1,1)).float().sum()
    accuracy = correct/y_train.size(0)

  if (epoch+1) % 10 == 0:
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss : {loss.item():.4f}, Accuracy: {accuracy.item() * 100:.2f}%')

Epoch [10/100], Loss : 0.5192, Accuracy: 88.79%
Epoch [20/100], Loss : 0.4036, Accuracy: 92.31%
Epoch [30/100], Loss : 0.3166, Accuracy: 93.41%
Epoch [40/100], Loss : 0.2536, Accuracy: 93.85%
Epoch [50/100], Loss : 0.2092, Accuracy: 94.51%
Epoch [60/100], Loss : 0.1778, Accuracy: 94.73%
Epoch [70/100], Loss : 0.1549, Accuracy: 95.16%
Epoch [80/100], Loss : 0.1377, Accuracy: 95.82%
Epoch [90/100], Loss : 0.1241, Accuracy: 96.26%
Epoch [100/100], Loss : 0.1132, Accuracy: 96.92%


**Model Evaluation**

In [40]:
# evaluation on training set
model.eval()
with torch.no_grad():
  outputs = model(X_train)
  predicted = outputs.round()
  correct = (predicted == y_train.view(-1,1)).float().sum()
  accuracy = correct/y_train.size(0)
  print(f'Accuracy on training data: {accuracy.item() * 100:.2f}%')

Accuracy on training data: 97.14%


In [41]:
# evaluation on test set
model.eval()
with torch.no_grad():
  outputs = model(X_test)
  predicted = outputs.round()
  correct = (predicted == y_test.view(-1,1)).float().sum()
  accuracy = correct/y_test.size(0)
  print(f'Accuracy on test data: {accuracy.item() * 100:.2f}%')

Accuracy on test data: 98.25%
